In [1]:
# REconocimiento facial utilizando el modelo Dlib a través
# de la libreria face_recognition 
# Dliib utiliztiene dos modos de operación utilizando una CNN o un HOG
# (Histograma de Gradientes Orientados) La CNN se recomienda utilizar cuando 
# Se ha instalado Dlib habilitando el uso de cuda este da mayor presición 
# EL HOG se recomienda cuando no se puede utilizar cuda 
import face_recognition
# Imutils es una libreria sobre open CV que facilita el uso de herramientas 
# como el rescalamiento, la rotación o la translación de las imágenes
import imutils
import pickle
import time
import cv2

In [2]:
# Cargamos la lista de rostros con su etiqueta que ya ha sido pasada por un 
# pre procesamiento en dlib el cual nos da un vector de 128 elementos que 
# son una representación numerica de la cara detectada
data = pickle.loads(open("./encoded_faces.pickle", "rb").read())

In [3]:
# Abrimos el archivo de video a evaluar, en este caso uno ya grabado 
# que es clip que contien partes de alguna peliculas de HP 
stream = cv2.VideoCapture("./examples/example_video_02.mp4")


In [4]:
# Iteramos por cada uno de los frames del video de entrada

(frameExists, frame) = stream.read()
# Creamos un writter el cual nos permitirá giuardar el nuevo video con las 
# caras detectas en disco
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter("./results/test-small-02.mp4", 
                    fourcc, 24, (frame.shape[1], frame.shape[0]), True)

while frameExists:
    # Open CV por defecto nos regresa la imagen en BGR en lugar de RGB 
    # Dlib funciona con las imagenes en formato RGB es por esto que tenemos
    # que hacer un cambio en formato de color, estge se realiza con ayuda de 
    # el mismo Open CV
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Le damos un nuevo Ancho al fram de 750 px para unificar el tamaño de los  
    # frames y de esta forma tener un tiempo de ejecución estandar con 
    # cualquier video que le pasemos 
    rgb = imutils.resize(rgb, width=750)
    # Sacamos un ratio de transformación para despues poder encuadrar
    # correctamente la cara en el video de salida
    r = frame.shape[1] / float(rgb.shape[1])

    # Detección de caras, con este método de face_recognition pasamos
    # la imagen por la red neuronal para poder detectar los recuadros
    # que encuadren cada cara, este método regresa una lista de puntos 
    # (arriba, derecha, abajo, izquierda)
    boxes = face_recognition.face_locations(rgb, model="hog")
    # Sacamos para cada cara detectada la serie principal de puntos, ojos,
    # nariz, boca, barbilla
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []

    # Para cada cara ya parametrizada detectada en el frame
    for encoding in encodings:
        # Buscamos cuales son las caras de entrenamiento con la menor
        # distancia para esto usamos np.linalg.norm y buscamos el menor
        # Este paso puede ser substituido por otra red neuronal, sin embargo,
        # tenemos que considerar la velocidad de esta comparación contra la de
        # la predicción de una red neuronal y evaluar la precisión 
        # para así poder determinar qué método cojnviene utilizar si se 
        # desea realizar analisis real-time de video.
        # El último parámetro que utilizamnos es la tolerancia que damos a 
        # la similitud de los rostros
        matches = face_recognition.compare_faces(data["encodings"], 
                                                 encoding,0.555)
        name = "Unknown"

        # Buscamos si hubo algún match en los rostros, de ser así
        # buscamos el nombre con el que más coincidencias haya tenido
        if True in matches:
            # Hacemos una lista de todos los indices que fueron un match
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            # Hacemos un conteo de cuales fueron las caras que hicieron match 
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # Buscamos la cara con mayor numero de repeticiones
            name = max(counts, key=counts.get)

        # Agregamos el nombre a la lista de nombres en caso de no haber match 
        # se ingresara Unknown
        names.append(name)

    # por cada rostro reconocido en el frame creamos un recuadro en la 
    # posición de la cara y lo pintamos de verde si hubo match, de rojo 
    # si no hubo
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # Hay que recordar que las coordenadas obtenidas son a partir de
        # una imagen escalad por lo que tenemos que rescalar las posiciones que
        # se obtuvieron con ayuda del ratio que sacamos con anterioridad
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)
        if name == 'Unknown':
            colour=(0,255,0)
        else:
            colour = (255,0,0)

        # Con ayuda de opencv pintamos el rectángulo sobre la cara 
        cv2.rectangle(frame, (left, top), (right, bottom), 
                      colour, 2)
        y = top - 15 if top - 15 > 15 else top + 15
        # y agreamos el nombere sobre el recuadro
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, colour, 2)
        
        # Escribimos nuestro nuevo cuadro a nuestro video de salida
        writer.write(frame)

    # Mosrtamos el nuevo cuadro en Tiempo real
    cv2.imshow("Harry Potter", frame)
    # Leemos el siguiente frame
    (frameExists, frame) = stream.read()

In [5]:
# Cerramos el stream
stream.release()
# Liberamos el Writer
writer.release()
cv2.destroyAllWindows()